<a href="https://colab.research.google.com/github/mostafa-ja/Anomaly-detection/blob/main/autoencoder6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [56]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):  #input_dim:features
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)
        print('outputs.shape = ',outputs.shape)
        print('hidden.shape = ',hidden.shape)
        print('cell.shape = ',cell.shape)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs,hidden, cell

In [57]:
model = Encoder(15,16,1)
model

Encoder(
  (rnn): LSTM(15, 16, batch_first=True)
)

In [58]:
input_data = torch.rand(1, 10, 15)
output,hidden, cell = model(input_data)

outputs.shape =  torch.Size([1, 10, 16])
hidden.shape =  torch.Size([1, 1, 16])
cell.shape =  torch.Size([1, 1, 16])


In [61]:
output[:,-1,:]== hidden

tensor([[[True, True, True, True, True, True, True, True, True, True, True,
          True, True, True, True, True]]])

In [64]:
hidden ==cell

tensor([[[False, False, False, False, False, False, False, False, False, False,
          False, False, False, False, False, False]]])

In [97]:
class Encoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers): #embedding dimensions
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=True,batch_first=True)


    def forward(self, src):

        #src = [src len, batch size]


        outputs, (hidden, cell) = self.rnn(src)
        print('outputs.shape = ',outputs.shape)
        print('hidden.shape = ',hidden.shape)
        print('cell.shape = ',cell.shape)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs[:,-1:,:]  #outputs[:,-1:,:]=output of the end of blocks which is concatenated of two directions(we use -1: for keeping the dimensions)

In [102]:
model = Encoder(15,32,1) #embedding dimensions
model

Encoder(
  (rnn): LSTM(15, 32, batch_first=True, bidirectional=True)
)

In [103]:
input_data = torch.rand(1, 10, 15)
output= model(input_data)
print(output.shape)

outputs.shape =  torch.Size([1, 10, 64])
hidden.shape =  torch.Size([2, 1, 32])
cell.shape =  torch.Size([2, 1, 32])
torch.Size([1, 1, 64])


In [91]:
input_data = torch.rand(1, 10, 15)
print(input_data[:,-1,:].shape)
print(input_data[:,-1:,:].shape)

torch.Size([1, 15])
torch.Size([1, 1, 15])


In [81]:
import numpy as np

# Create a vector of shape [1, 5]
vector = np.array([[1, 2, 3, 4, 5]])

# Repeat the vector along the second dimension to create a tensor of shape [1, 10, 5]
tensor = np.tile(vector, (1, 10, 1))

# Check the shape of the resulting tensor
print(tensor.shape)
print(tensor)


(1, 10, 5)
[[[1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]
  [1 2 3 4 5]]]


In [85]:
class Decoder(nn.Module):
    def __init__(self, input_dim, hid_dim, n_layers):       #input _dim depends on encoder's output
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers

        self.rnn = nn.LSTM(input_dim, hid_dim, n_layers, bidirectional=False,batch_first=True)

    def forward(self, src):

        #src = [src len, batch size]
        src = torch.tile(src, (1, 10, 1)) # give the output of encoder to each cell of decoder

        outputs, (hidden, cell) = self.rnn(src)
        print('outputs.shape = ',outputs.shape)
        print('hidden.shape = ',hidden.shape)
        print('cell.shape = ',cell.shape)

        #outputs = [batch size,src len, hid dim * n directions]
        #hidden = [batch size,n layers * n directions,  hid dim]
        #cell = [batch size, n layers * n directions, hid dim]

        #outputs are always from the top hidden layer

        return outputs

In [106]:
model = Decoder(64,15,1)
model

Decoder(
  (rnn): LSTM(64, 15, batch_first=True)
)

In [107]:
input_data = torch.rand(1, 1, 64)
output= model(input_data)
print(output.shape)

outputs.shape =  torch.Size([1, 10, 15])
hidden.shape =  torch.Size([1, 1, 15])
cell.shape =  torch.Size([1, 1, 15])
torch.Size([1, 10, 15])


In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src):
        encoded, _ = self.encoder(src)
        decoded, _ = self.decoder(encoded)
        print(encoded.shape)
        print(decoded.shape)
        return decoded

In [ ]:
model = Autoencoder(64,15,1)
model

In [ ]:
input_data = torch.rand(1, 1, 64)
output= model(input_data)
print(output.shape)

In [37]:
import torch
import torch.nn as nn

class LSTMAutoencoder(nn.Module):

  def __init__(self, input_size, hidden_size, num_layers=1):
    super(LSTMAutoencoder, self).__init__()
    self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=False)

  def forward(self, inputs):
    a,(b,c) = self.encoder(inputs)
    print(a.shape,b.shape,c.shape)
    return a,b,c

In [4]:

import torch
import os
import numpy as np
import pandas as pd
import torch.nn.functional as F
import torch.nn as nn

In [30]:
class Encoder(nn.Module):
    def __init__(self, seq_len, n_features,embedding_dim,n_layers):
        super(Encoder, self).__init__()

        self.seq_len = seq_len
        self.n_features = n_features
        self.embedding_dim = embedding_dim
        self.hidden_dim = 2 * embedding_dim

        self.rnn1 = nn.LSTM(
          input_size=n_features,
          hidden_size=self.hidden_dim,
          num_layers=n_layers,
          batch_first=True  # True = (batch_size, seq_len, n_features)
                            # False = (seq_len, batch_size, n_features)
                            #default = false
        )
        self.rnn2 = nn.LSTM(
          input_size=self.hidden_dim,
          hidden_size=embedding_dim,
          num_layers=n_layers,
          batch_first=True
        )
        self.linear = nn.Linear(embedding_dim,8)
    def forward(self, x):
        # (batch, seq, feature)   #(1,4,1)
        a,(b,c) = self.rnn1(x)
        print(a.shape,b.shape,c.shape)
        x, (_, _) = self.rnn1(x) #(1,4,256)
        x, (hidden_n, _) = self.rnn2(x)
        x = self.linear(x)
        # x shape (1,4,128)
        # hidden_n (1,1,128)
        return x

In [31]:
model = Encoder(10,15,16,2)
model

Encoder(
  (rnn1): LSTM(15, 32, num_layers=2, batch_first=True)
  (rnn2): LSTM(32, 16, num_layers=2, batch_first=True)
  (linear): Linear(in_features=16, out_features=8, bias=True)
)

In [32]:
input_data = torch.rand(1, 10, 15)
output = model(input_data)
output.shape

torch.Size([1, 10, 32]) torch.Size([2, 1, 32]) torch.Size([2, 1, 32])


torch.Size([1, 10, 8])

In [ ]:
class Decoder(nn.Module):
    def __init__(self, seq_len, n_features,embedding_dim,n_layers):
        super(Decoder, self).__init__()

        self.seq_len = seq_len
        self.hidden_dim = 2 * embedding_dim
        self.n_features = n_features

        self.rnn1 = nn.LSTM(
        input_size=embedding_dim,
        hidden_size=embedding_dim,
        num_layers=n_layers,
        batch_first=True
        )
        self.rnn2 = nn.LSTM(
        input_size=embedding_dim,
        hidden_size=self.hidden_dim,
        num_layers=n_layers,
        batch_first=True
        )
        self.output_layer = nn.Linear(self.hidden_dim, n_features)

    def forward(self, x):

        x, (hidden_n, cell_n) = self.rnn1(x) #(1,4,128)
        x, (hidden_n, cell_n) = self.rnn2(x) #(1,4,256)
        x = x.reshape((self.seq_len, self.hidden_dim)) #(4,256)

        return self.output_layer(x) # (4,1)

In [ ]:
import torch
import torch.nn as nn

class LSTMAutoencoder(nn.Module):

  def __init__(self, input_size, hidden_size, num_layers=2):
    super(LSTMAutoencoder, self).__init__()
    self.encoder = nn.LSTM(input_size, hidden_size, num_layers=num_layers, bidirectional=True)
    self.decoder = nn.LSTM(hidden_size * 2, input_size, num_layers=num_layers)

  def forward(self, inputs):
    encoded, _ = self.encoder(inputs)
    decoded, _ = self.decoder(encoded)
    return decoded

